In [ ]:
import numpy as np
import keras
from keras import layers
from keras import regularizers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras import optimizers,losses
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
import graphviz
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import h5py

import keras.backend as K
# 训练数组表示样本通道的分量是最后一个
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [ ]:
# 数据文件

file = r'QKT_week1_cwt.h5'

In [ ]:
# 读取训练和测试数据，验证地震事件和噪声窗口的数组结构

dataset = h5py.File(file, "r")

event_data = np.array(dataset["event"][:]) 
noise_data = np.array(dataset["noise"][:]) 

print("event_data shape: " + str(event_data.shape))
print("noise_data shape: " + str(noise_data.shape))

In [ ]:
# 读取训练数据，验证训练数据集地震事件和噪声窗的数据结构
# n为训练集地震事件和噪声数量，整个训练集的样本数为2n

n = 800

trainquake = event_data[0:n,:,:]
trainnoise = noise_data[0:n,:,:]

train_set_x = np.concatenate((trainquake,trainnoise), axis = 0)

label_e = [1]*n
label_n = [0]*n
label_e.extend(label_n)

train_set_y = np.array(label_e)
train_set_y = train_set_y.reshape((-1,1))

print("number of training examples = " + str(train_set_x.shape[0]))
print("X_train shape: " + str(train_set_x.shape))
print("Y_train shape: " + str(train_set_y.shape))

In [ ]:
# 读取测试数据，验证测试数据集地震事件和噪声窗的数据结构
# m为测试集地震事件和噪声数量，整个测试集的样本数为2m

m = 300

test_set_x_quake = event_data[1000:1000+m,:,:]
test_set_x_noise = noise_data[1000:1000+m,:,:]

test_set_y_quake = np.array([1]*m)
test_set_y_noise = np.array([0]*m)

test_set_y_qukae = test_set_y_quake.reshape((-1,1))
test_set_y_noise = test_set_y_noise.reshape((-1,1))

print("X_test quake shape: " + str(test_set_x_quake.shape))
print("Y_test quake shape: " + str(test_set_y_quake.shape))
print("X_test noise shape: " + str(test_set_x_noise.shape))
print("Y_test noise shape: " + str(test_set_y_noise.shape))

In [ ]:
# 轻量化CNN模型的定义和训练

def CCLSN_c2f1(input_shape):
    
    X_input = Input(input_shape)
    
    #conv1 
    #CONV -> BN -> RELU
    Z1 = Conv2D(12, 5, strides=(1,2), name="conv1",padding="SAME")(X_input)
    bn1 = BatchNormalization(name="bn1",axis=3)(Z1)
    A1 = Activation("relu")(bn1)
    #MAXPOOL
    pool1 = MaxPooling2D((2, 2),name="maxpool1")(A1)
    
    #conv2
    Z2 = Conv2D(12, 5, strides=(1, 2), name="conv2",padding="SAME")(pool1)
    bn2 = BatchNormalization(name="bn2",axis=3)(Z2)
    A2 = Activation("relu")(bn2)
    pool2 = MaxPooling2D((1, 2),name="maxpool2")(A2)
    
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    flaten1 = Flatten()(pool2)
    #kernel_regularizer=regularizers.l2(0.01),
    Z3 = Dense(1, activation="sigmoid",  name="fc")(flaten1)
    
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs=X_input, outputs=Z3, name="NCLSN_c2f1")
    
    ### END CODE HERE ###
    
    return model

Model = CCLSN_c2f1((45,251,3))

Model.compile(optimizer = "Adam", loss="binary_crossentropy", metrics=['accuracy'])

#EarlyStopping = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.003, patience=1)
EarlyStopping = keras.callbacks.EarlyStopping(monitor='acc', min_delta=0.005, patience=1)

#batch_size=100
Model.fit(x=train_set_x,y=train_set_y, callbacks=[EarlyStopping], batch_size=200, epochs=15)

In [ ]:
# 模型精度测试

preds_e = Model.evaluate(x=test_set_x_quake,y=test_set_y_quake)
preds_n = Model.evaluate(x=test_set_x_noise,y=test_set_y_noise)

print("Test event Accuracy = " + str(preds_e[1]))
print("Test noise Accuracy = " + str(preds_n[1]))

In [ ]:
# 模型结构总结

Model.summary()

In [ ]:
# 模型保存

'''
save方法保存模型的全部信息
包括：模型的结构，以便重构该模型、模型的权重、训练配置（损失函数，优化器等）、优化器的状态，以便于从上次训练中断的地方开始
model.save('****.h5')
通过keras.models.load_model(filepath, custom_objects=None, compile=True)实例化模型

model.to_json()或者model.to_yaml()
只保存模型的结构（到json或者yaml格式），不包含权重或配置信息。返回一个json_string或者yaml_string。
json_string = model.to_json()
yaml_string = model.to_yaml()
通过keras.models.model_from_json(json_string)或者keras.models.model_from_yaml(yaml_string)来载入模型（不包含权重和配置信息）。

'''

CCLSN_c2f1.save('CCLSN_c2f1.h5')

In [ ]:
# 模型结构绘图和保存

plot_model(QuakeModel, to_file='./CCLSN_c2f1.png')
SVG(model_to_dot(QuakeModel).create(prog='dot', format='svg'))